Tensor

In [ ]:
import torch
import numpy as np

x = torch.Tensor([[1, 2], [3, 4]])
x = torch.from_numpy(np.array([[1, 2], [3, 4]]))
print(f"pytorch \n {x}")


x = np.array([[1, 2], [3, 4]])
print(f"numpy \n {x}")

Autograd

In [ ]:
import torch
x = torch.FloatTensor(2, 2)
y = torch.FloatTensor(2, 2)
y.requires_grad_(True)

z = (x + y) + torch.FloatTensor(2, 2)

기울기를 구할 필요가 없는 연산의 경우 with 문법을 사용하여 연산을 수행한다.
- with 문법은 컨텍스트 관리자를 사용하여 자원을 효율적으로 관리하기 위해 사용한다.

In [ ]:
x = torch.FloatTensor(2, 2)
y = torch.FloatTensor(2, 2)
y.requires_grad_(True)

with torch.no_grad():
    z = (x + y) + torch.FloatTensor(2, 2)

피드 포워드
- 선형 계층(Linear Layer) 또는 완전연결계층(Fully-connected Layer) 구현
- 아래의 코드는 역전파 알고리즘을통한 학습은 하지 못한다.

In [ ]:
def linear(x, W ,b):
    y = torch.mm(x, W) + b
    # mm = matrix multiplication => pytorch에서 제공하는 행렬 곱셈 연산자
    return y

x = torch.FloatTensor(16, 10)
W = torch.FloatTensor(10, 5)
b = torch.FloatTensor(5)

y = linear(x, W, b)

nn.Module
- nn.Module 클래스를 이용하여 사용자가 그 위에서 필요한 모델 구조를 구현할 수 있도록 한다.

In [ ]:
import torch
import torch.nn as nn

class MyLinear(nn.Module): # MyLinear 클래스가 nn.Module 클래스를 상속받는다.

    def __init__(self, input_size, output_size):
        super().__init__()

        self.W = torch.FloatTensor(input_size, output_size)
        self.b = torch.FloatTensor(output_size)

    def forward(self, x):
        y = torch.mm(x, self.W) + self.b

        return y

In [ ]:
x = torch.FloatTensor(16, 10)
linear = MyLinear(10, 5)
y = linear(x)

parameters()
- 모듈 내에 선언된 학습이 필요한 파라미터들을 반환하는 iterator
- 위의 코드로 진행하면, linear 모듈 내에는 학습 가능한 파라미터가 없다.
    - Parameter라는 클래스를 사용하여 텐서를 감싸서 위의 문제를 해결한다.

In [ ]:
params = [p.size() for p in linear.parameters()]
print(params)

In [ ]:
import torch
import torch.nn as nn

class MyLinear(nn.Module): # MyLinear 클래스가 nn.Module 클래스를 상속받는다.

    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__()

        self.W = nn.Parameter(torch.FloatTensor(input_size, output_size), requires_grad=True)
        self.b = nn.Parameter(torch.FloatTensor(output_size), requires_grad=True)

    def forward(self, x):
        y = torch.mm(x, self.W) + self.b

        return y
    
x = torch.FloatTensor(16, 10)
linear = MyLinear(10, 5)
y = linear(x)

In [ ]:
params = [p.size() for p in linear.parameters()]
print(params)

위의 코드 정리

In [ ]:
class MyLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__() # 파이썬 2와 호환하기 위한 코드 파이썬 3에서는 다음과 같이 작성한다. super().__init__()
        
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        y = self.linear(x)

        return y

In [ ]:
linear = MyLinear(10, 5)
print(linear)

역전파 수행
- 피드포워드를 통해 얻은 값에서 실제 정답값과의 차이를 계산하여 오류(손실)를 뒤로 전달(back propagation)한다.

In [ ]:
objective = 100

x = torch.FloatTensor(16, 10)
linear = MyLinear(10 ,5)
y = linear(x)
loss = (objective - y.sum())**2

loss.backward()

train(), eval()
- nn.Module을 상속받아 구현하고 생성한 객체는 기본적으로 훈련 모드이다.
- 이를 eval()을 사용하여 추론 모드로 바꿔주면, dropout 또는 배치 정규화(batch-normalization)와 같은 학습과 추론 시 서로 다른 forward() 동작을 하는 모듈들에 대해서도 각 상황에 따라 올바르게 동작한다.
    - 추론이 끝나면 다시 train()을 선언하여 원래의 훈련 모드로 변경해야 한다.

In [ ]:
# Training
linear.eval()
# Do some inference process
linear.train()
# Restart training again

선형회귀분석 예제
- 임의로 생성한 텐서들을
- 근사하고자 하는 정답 함수에 넣어 정답(y)를 구하고
- 그 정답과 신경망을 통과한 y_hat과의 차이를 MSE를 통해 구하고
- SGD(Stochastic Gradient Descent)를 통해 최적화한다.

In [ ]:
# 1. 1개의 선형 계층을 가진 MyModel 모듈 선언
import random
import torch
import torch.nn as nn

class MyModel(nn.Module):

    def __init__(self, input_size, output_size):
        super(MyModel, self).__init__()

        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        y = self.linear(x)
        return y


In [ ]:
# 2. 임의의 함수 동작 구현
def ground_truth(x):
    return 3 * x[:, 0] + x[:, 1] - 2 * x[:,2]

In [ ]:
# 3. 모델과 텐서를 입력받아 피드포워딩한 후, 역전파 알고리즘을 수행하여 경사하강법의 한 스텝을 수행한다.
def train(model, x, y, optim):
    # initialize gradients in all parameters in module
    optim.zero_grad()

    # feed-forward
    y_hat = model(x)
    # get error between answer and inferenced
    loss = ((y - y_hat)**2).sum() / x.size(0)

    # back-propagation
    loss.backward()

    # one-step of gradient descent
    optim.step()

    return loss.data

In [ ]:
# 3-1. 앞에서 만든 함수들을 사용하기 위해 하이퍼 파라미터 설정
batch_size = 1
n_epochs = 1000
n_iter = 10000

model = MyModel(3, 1)
optim = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.1)

print(model)

In [ ]:
# 4. 위의 값들을 이용하여 평균 손실값이 0.001보다 작아질 때까지 훈련한다.
for epoch in range(n_epochs):
    ave_loss = 0

    for i in range(n_iter):
        x = torch.rand(batch_size, 3)
        y = ground_truth(x.data)

        loss = train(model, x, y, optim)

        ave_loss += loss
    avg_loss = ave_loss / n_iter
    
    # simple test sample to check the network
    x_valid = torch.FloatTensor([[.3, .2, .1]])
    y_valid = ground_truth(x_valid.data)

    model.eval()
    y_hat = model(x_valid)
    model.train()

    print(avg_loss, y_valid.data[0], y_hat.data[0, 0])

    if avg_loss < 0.001: # finish the training if the loss is smaller than 0.001
        break

Use GPU

In [ ]:
# Note that tensor is declared in torch.cuda.
x = torch.cuda.FloatTensor(16, 10)
linear = MyLinear(10, 5)
# .cuda() let module move to GPU memory
linear.cuda()
y = linear(x)